In [3]:
import pandas as pd
import datetime as dt
import pandas as pd, numpy as np, datetime
import jupyterNotebookModuleImport
import plotly
import plotly.plotly as py
import plotly.graph_objs as go 

import plotly
from plotly.graph_objs import Scatter, Layout

import sqlite3
import cufflinks as cf
import matplotlib.pyplot as plt


In [4]:
priceData = pd.read_csv('priceData.csv', sep='\t',index_col=0)
earningsData = pd.read_csv('earningsData.csv', sep='\t',index_col=0)
stockMetaData = pd.read_csv('stockMetaData.csv', sep='\t',index_col=0)

In [5]:
#pd.to_datetime(earningsData['date'])
earningsData['dtString'] = earningsData['date'] + ' ' + earningsData['announce']
earningsData['dt'] = pd.to_datetime(earningsData['dtString'], format='%Y-%m-%d %H:%M')
earningsData = earningsData.drop(['dtString'],axis=1)
priceData.index = pd.to_datetime(priceData.index, format='%Y-%m-%d %H:%M')


In [6]:
def returnChartData(symbol, offsetBefore = -7, offsetAfter = 14, eventNumber = 0):
    plotly.offline.init_notebook_mode(connected=True)
    
    dfEvent = earningsData[earningsData['Symbol']==symbol].iloc[eventNumber]
    dt = dfEvent['dt']
    start = dt + datetime.timedelta(days=offsetBefore)
    end = dt + datetime.timedelta(days=offsetAfter)
    priceData_chart = priceData[(priceData.index > start) & (priceData.index <= end)]
    priceData_chart = priceData_chart[priceData_chart['Symbol']==symbol]
    df = priceData_chart
    
    temp = dfEvent[['EPS Estimate', 'Reported EPS', 'Surprise(%)', 'date']].transpose()
    temp.index = temp.index + ': '
    txt = str(temp).replace('  ','')
    eventDataWithAllTimestamps = priceData_chart.copy()
    eventDataWithAllTimestamps['Bar'] = None
    eventDataWithAllTimestamps.iloc[eventDataWithAllTimestamps.index.get_loc(dt,method='pad'),eventDataWithAllTimestamps.columns.get_loc('Bar')] = 1
    eventDataWithAllTimestamps['BarMax_vol'] = eventDataWithAllTimestamps['Bar'] * df.Vol.max() 
    eventDataWithAllTimestamps['BarMin_vol'] = eventDataWithAllTimestamps['Bar'] * df.Vol.min() 
    eventDataWithAllTimestamps['BarMax_price'] = eventDataWithAllTimestamps['Bar'] * (df.Close.max() - df.Close.min())
    eventDataWithAllTimestamps['BarMin_price'] = eventDataWithAllTimestamps['Bar'] * df.Close.min() 
    eventDataWithAllTimestamps['Txt'] = txt
    
    marketCap = stockMetaData[stockMetaData['Symbol'] == symbol].head(1)['Market Cap'].iloc[0]
    EPS_expected = dfEvent['EPS Estimate']
    EPS = dfEvent['Reported EPS']
    surprise = dfEvent['Surprise(%)'] + "%"
    title = symbol + ' | Cap: ' + marketCap + ' | EPS(expected): ' + str(EPS) +'(' + str(EPS_expected) + ') | Surprise: '+str(surprise)
    
    return df, eventDataWithAllTimestamps, title

In [7]:
symbolList = stockMetaData.sort_values(by=['Market Cap'], ascending=False).head(20)

buttonList = []

for i,r in symbolList.iterrows():
    df, ev, title = returnChartData(r['Symbol'])
    d = dict(
                args=[{'x':[df.index,df.index,ev.index], 'y':[df.Close,df.Vol,ev.BarMax_vol]},
                      {'xaxis.type': 'category', 'title': title}
                     ],
                label=r['Symbol'],
                method='update'
            )
    buttonList.append(d)


updatemenus=list([
    dict(
        buttons=list(buttonList),
        direction = 'down',
        pad = {'r': 0, 't': 0},
        showactive = True,
        x = 0,
        xanchor = 'left',
        y = 1,
        yanchor = 'top'
    ),
])

In [9]:

symbol = 'GOOGL'

df, ev, title = returnChartData(symbol)

trace = go.Scatter(x=df.index, y=df.Close, mode = 'lines+markers', marker=dict(size=3))
trace_vol = go.Bar(x=df.index, y=df.Vol)
trace_test = go.Bar(x=ev.index, y=ev.BarMax_vol)
#trace_event_price = go.Bar(x=ev.index, y=ev.BarMax_price, base=ev.BarMin_price)

fig = plotly.tools.make_subplots(rows=2, cols=1 , shared_xaxes=True) 
fig.append_trace(trace, 1, 1)
fig.append_trace(trace_test, 2, 1)
fig.append_trace(trace_vol, 2, 1)
#fig.append_trace(trace_event_price, 1, 1)

fig['layout']['xaxis1'].update(type='category')
fig['layout']['xaxis2'].update(type='category')
fig['layout']['yaxis1'].update(domain=[0.2,1]) #
fig['layout']['yaxis2'].update(domain=[0,0.2])

fig['layout'].update(showlegend=False, title=title)
fig['layout'].update(updatemenus = updatemenus)

plotly.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%),date,announce,dt
0,ABMD,Abiomed Inc,Before Market Open,0.51,0.29,-43.14,2018-02-01,0:00,2018-02-01 00:00:00
1,BABA,Alibaba Group Holding Ltd,Before Market Open,1.67,1.67,-,2018-02-01,0:00,2018-02-01 00:00:00
3,NCZ,Allianzgi Convertible & Income Fund II,After Market Close,-,-,-,2018-02-01,23:59,2018-02-01 23:59:00
4,GOOGL,Alphabet Inc,After Market Close,9.98,9.7,-2.81,2018-02-01,23:59,2018-02-01 23:59:00
5,MO,Altria Group Inc,Before Market Open,0.8,0.91,13.75,2018-02-01,0:00,2018-02-01 00:00:00


In [23]:
df = priceData.copy()
df['dt'] = df.index.values
dfEvent = earningsData.copy()

In [10]:
#df = priceData[priceData['Symbol'].isin(['ABMD','AAPL','GOOGL'])].copy()
#df['dt'] = df.index.values
#dfEvent = earningsData[earningsData['Symbol'].isin(['ABMD','AAPL','GOOGL'])]

In [179]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
dfEvent.to_sql('dfEvent', conn, index=False)
df.to_sql('df', conn, index=False)

In [181]:
qry = '''
    select  
        dfEvent.dt event_dt,
        dfEvent.Symbol Symbol,
        dfEvent.Company,
        dfEvent.[Earnings Call Time],
        dfEvent.[EPS Estimate],
        dfEvent.[Reported EPS],
        dfEvent.[Surprise(%)],
        df.dt price_dt,
        df.Close
    from
        dfEvent join df 
        on df.dt between DATE(dfEvent.dt, '-14 DAYS') and DATE(dfEvent.dt, '+30 DAYS')
        and df.Symbol = dfEvent.Symbol
    order by 
        dfEvent.Symbol,
        df.dt
    '''
df_merged_sql = pd.read_sql_query(qry, conn)

In [183]:
#Find where the events are and index them from -x to +z where the event date is index=0, allowing for time normalisation for charting and averaging

df_merged = df_merged_sql.copy()

#apply the index from 0 onwards for each event / symbol group (indexer)
df_merged['indexer'] = df_merged.groupby(['event_dt','Symbol']).cumcount()

#calculate the seconds difference from the event (absolute), to calculate the closest value
df_merged['price_dt'] = pd.to_datetime(df_merged['price_dt'])
df_merged['event_dt'] = pd.to_datetime(df_merged['event_dt'])
df_merged['datediff']  = abs(df_merged['event_dt'] - df_merged['price_dt']).astype('timedelta64[s]')
df_merged['is_closest'] = False
df_merged.loc[df_merged.groupby(['event_dt','Symbol'])['datediff'].idxmin(),'is_closest'] = True

#get a dataframe of just the events and the "indexer" value of these events. These will be subtracted form indexer to normalise.
df_merged_events = df_merged[df_merged['is_closest']==True][['event_dt','Symbol','indexer','Close']]
df_merged_events.rename(columns={'indexer': 'indexer_event','Close': 'Close_event'}, inplace=True)

#merge the data with the events dataframe and subtract the indexer's to normalise
df_merged_withevent = df_merged.merge(df_merged_events, left_on=['event_dt','Symbol'], right_on=['event_dt','Symbol'], how='inner')
df_merged_withevent['eventIndex'] = df_merged_withevent['indexer'] - df_merged_withevent['indexer_event']

#normalise the price into a percentage
df_merged_withevent['pricePercent'] = (df_merged_withevent['Close'] - df_merged_withevent['Close_event']) / df_merged_withevent['Close_event']

#add the grouping ie symbol_event
df_merged_withevent["Symbol_event"] = df_merged_withevent['Symbol'] + ': ' + df_merged_withevent['event_dt'].dt.strftime('%Y-%m-%d %H')

In [193]:
df_merged_withevent.head(2)

,event_dt,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%),price_dt,Close,indexer,datediff,is_closest,indexer_event,Close_event,eventIndex,pricePercent,Symbol_event
0,2018-02-14 23:59:00,A,Agilent Technologies Inc,After Market Close,0.58,0.66,13.79,2018-01-31 10:30:00,73.77,0,1258140.0,False,145,69.62,-145,0.059609,A: 2018-02-14 23
1,2018-02-14 23:59:00,A,Agilent Technologies Inc,After Market Close,0.58,0.66,13.79,2018-01-31 11:00:00,73.82,1,1256340.0,False,145,69.62,-144,0.060327,A: 2018-02-14 23


In [227]:
df_pivot_SymbolEvents = df_merged_withevent.pivot(index='eventIndex',columns='Symbol_event',values='pricePercent')
nullsReduced = df_pivot_SymbolEvents[df_pivot_SymbolEvents.isnull().sum(axis=1)<50]
df_merged_withevent_reducedNulls = df_merged_withevent[df_merged_withevent.eventIndex.isin(nullsReduced.index)].copy()

In [247]:
#df_merged_withevent_reducedNulls.head()

In [244]:
df_merged_withevent_reducedNulls['Surprise(%)'] = pd.to_numeric(df_merged_withevent_reducedNulls['Surprise(%)'], errors='coerce')


In [245]:
bins = [-50,-20,-10,-5,0,5,10,20,50]
df_merged_withevent_reducedNulls['Surprise_Bin'] = pd.cut(df_merged_withevent_reducedNulls['Surprise(%)'], bins)

In [248]:
df_merged_withevent_reducedNulls.head(2)

,event_dt,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%),price_dt,Close,indexer,datediff,is_closest,indexer_event,Close_event,eventIndex,pricePercent,Symbol_event,Surprise(%)_num,Surprise_Bin
34,2018-02-14 23:59:00,A,Agilent Technologies Inc,After Market Close,0.58,0.66,13.79,2018-02-02 14:30:00,72.13,34,1070940.0,False,145,69.62,-111,0.036053,A: 2018-02-14 23,13.79,"(10, 20]"
35,2018-02-14 23:59:00,A,Agilent Technologies Inc,After Market Close,0.58,0.66,13.79,2018-02-02 15:00:00,71.99,35,1069140.0,False,145,69.62,-110,0.034042,A: 2018-02-14 23,13.79,"(10, 20]"


In [253]:
#dfPlot = df_merged_withevent_reducedNulls.pivot(index='eventIndex',columns='Surprise_Bin',values='pricePercent', aggfunc=np.mean)
dfPlot = pd.pivot_table(df_merged_withevent_reducedNulls, values='pricePercent', index=['eventIndex'],columns=['Surprise_Bin'], aggfunc=np.mean)

In [198]:
dfPlot = df_merged_withevent_reducedNulls.groupby('eventIndex').agg({'pricePercent':
                                  {
                                   'perc25': lambda x: np.percentile(x, 25),
                                    'perc40': lambda x: np.percentile(x, 40),
                                    'Mean': np.mean,
                                    'perc60': lambda x: np.percentile(x, 60),
                                   'perc75': lambda x: np.percentile(x, 75)
                                  
                                  }})

In [255]:
dfPlot.iplot(kind='scatter', filename='cufflinks/cf-simple-line',colorscale='accent')